### Sample program for LSTM (N-d input/output)  
https://oku.edu.mie-u.ac.jp/~okumura/stat/160118.html  

#### Import libraries  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

#### Parameters  

In [ ]:
np.random.seed(11)
tf.random.set_seed(11)

# To show more rows and columns without "..."
pd.options.display.max_columns=999
pd.options.display.max_rows=999

#### Read CSV file  

In [ ]:
# read data
df = pd.read_csv('weather_in_tokyo_and_icecream_in_japan-utf8.csv',
                 delimiter=',',skiprows=6, header=0)
print(df.shape)
print(df.info())
display(df.head())

#### Convert datatype of Month column to 'datetime'  

In [ ]:
df['Month'] = pd.to_datetime(df['Month'])
print(df.info())
display(df.head())

#### Time series plot  

In [ ]:
plt.plot(df['Month'], df['y1'])
plt.xlabel('Month')
plt.ylabel('Icecream')
plt.show()

#### Set parameters for training / prediction  

In [ ]:
w = 24
n_out = 12
n_test = 12

#### Standardization of training data  

In [ ]:
sc = MinMaxScaler(feature_range=(0, 1), copy=True)
xy = df.drop(columns='Month')
sc.fit(xy[:-n_test])
xy_std = sc.transform(xy)
print(xy_std.shape)

#### Prepare for train data and teacher signal  

In [ ]:
weather = xy_std[:, :-1]
print(weather.shape)
display(weather[:5])

In [ ]:
icecream = xy_std[:, -1]
print(icecream.shape)
print(icecream[:5])

#### Prepare for windows of data  

In [ ]:
x = []
y = []

n_months = icecream.shape[0]

for i in range(0, n_months-w-n_out):
    x.append(weather[i:i+w])
    y.append(icecream[i+w:i+w+n_out])
x = np.array(x)
y = np.array(y)
print(x.shape)  # debug
print(x[:3])
print(y.shape)  # debug
print(y[:3])

#### Model building  

In [ ]:
n_hidden = 20

# early stopping and model checkpoint
es_cb = EarlyStopping(monitor='val_loss',
                      patience=10, verbose=1, mode='auto')

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(x.shape[1],x.shape[2]),
               return_sequences=False))
model.add(Dense(n_out, activation='linear'))

model.summary()

In [ ]:
model.compile(loss="mean_squared_error", optimizer='adam')

#### Training (learning)  

In [ ]:
%%time

# Learning 
n_epochs = 500
val_split = 0.2
batch_size = 1

fit_log = model.fit(x[:-n_test], y[:-n_test], batch_size=batch_size, epochs=n_epochs,
                    validation_split=val_split, callbacks=[es_cb])

#### Do prediction  

In [ ]:
# Against training data
pred = model.predict(x[:-n_test])
print(pred.shape)  # debug

In [ ]:
# Future prediction
future_test = x[-n_test:]
print(future_test.shape)
print(future_test[:3])

In [ ]:
future_result = model.predict(future_test)
print(future_result.shape)

#### Generalization performance  

In [ ]:
mse = mean_squared_error(y[-n_test:], future_result)
print('mse=', mse)

#### Plot of the results  

In [ ]:
for i in range(n_test):
    plt.plot(range(i,i+n_out), y[i-n_test])
    plt.plot(range(i,i+n_out), future_result[i])
    plt.xlabel('Month')
    plt.ylabel('Icecream')
    plt.show()

In [ ]:
for i in range(n_test):
    plt.plot(range(i,i+n_out), y[i-n_test], c='b')
    plt.plot(range(i,i+n_out), future_result[i], c='r')
plt.xlabel('Month')
plt.ylabel('Icecream')
plt.show()

#### Plot of the log during the training  

In [ ]:
# Plot Training loss & Validation Loss
loss = fit_log.history["loss"]
val_loss = fit_log.history["val_loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, label="loss" )
plt.plot(epochs, val_loss, label="val_loss")
plt.title("Training and Validation loss")
plt.xlabel('Epoch')
plt.ylabel('Loss / Val_loss')
plt.legend()
plt.show()